# Classification

In [1]:
# Ici nous allons tester des methodes de classifications 

In [2]:
# Librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import scipy
import scipy.stats
import time
import tensorflow as tf

# Machine learning

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

In [3]:
# Etiquetage de chaque colonne 
index = ['Fichier', 'Score', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18']

# Légende
#'S1':'Largeur moyenne', 
#'S2':'Écart type sur la largeur',
#'S3':'Envergure moyenne',
#'S4':'Écart type sur l’envergure',
#'S5':'Profondeur moyenne',
#'S6':'Écart-type sur la profondeur',
#'S7':'Longueur de la première et de la dernière arête',
#'S8':'Longueur moyenne de la plus grande arête',
#'S9':'Longueur de la plus grande arête de chaque tournée, divisée par la longueur de la tournée',
#'S10':'Longueur de la plus grande arête intérieure de chaque tournée (arête non connectée au dépôt), divisée par la longueur de la tournée',
#'S11':'Longueur moyenne de la première et de la dernière arête de chaque tournée',
#'S12':'Demande du premier et dernier client de chaque tournée, divisée par la charge du véhicule',
#'S13':'Demande du client le plus éloigné du dépôt, pour chaque tournée, divisée par la charge du véhicule',
#'S14':'Écart type sur la demande du client le plus éloigné du dépôt',
#'S15':'Écart type sur la longueur de chaque tournée',
#'S16':'Distance (euclidienne) moyenne entre les centre de gravité tournées',
#'S17':'Écart type sur le nombre de client de chaque tournée',
#'S18':'Degré de chaque voisinage moyen des clients'.

# Regroupement par fichier. 
keys1 = ["01", "02", "03", "04", "05", "06", "07", "08", "09","10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27"]
keys2 = ["01", "02", "03", "04", "05", "06", "07", "08", "09","10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26"]

# Concatenation de tous les fichiers. 
files1 = glob.glob("HGS/2113/XML100_2113_*.csv")
files2 = glob.glob("HGS/2213/XML100_2213_*.csv")
files3 = glob.glob("HGS/3113/XML100_3113_*.csv")
files4 = glob.glob("HGS/3213/XML100_3213_*.csv")

data_center_random = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files1], keys = keys1)
data_center_cluster = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files2], keys = keys1)
data_coin_random = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files3], keys = keys2)
data_coin_cluster = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files4], keys = keys2)

In [4]:
# Suppression de la colonne "LS7" car ne contient que des valeurs nulles.
# Suppression de la colonne "Fichier" qui ne donne aucune information sur le score. 

data_center_random = data_center_random.drop(['Fichier', 'S7'], axis=1)
data_center_cluster = data_center_cluster.drop(['Fichier', 'S7'], axis=1)
data_coin_random = data_coin_random.drop(['Fichier', 'S7'], axis=1)
data_coin_cluster = data_coin_cluster.drop(['Fichier', 'S7'], axis=1)

In [5]:
A = data_center_random['Score'].describe(percentiles = [0.2, 0.5, 0.8])
B = data_center_cluster['Score'].describe(percentiles = [0.2, 0.5, 0.8])
C = data_coin_random['Score'].describe(percentiles = [0.2, 0.5, 0.8])
D = data_coin_cluster['Score'].describe(percentiles = [0.2, 0.5, 0.8])

data = {'Center random': A, 'Center cluster': B, 'Coin random': C, 'Coin cluster': D}
pd.DataFrame(data, 
            index = ['count', 'mean', 'std', 'min', '20%', '50%', '80%', 'max'])

,Center random,Center cluster,Coin random,Coin cluster
count,2.536722e+06,3.302699e+06,4.137693e+06,4.132400e+06
mean,1.383565e+04,1.073829e+04,2.175919e+04,1.999346e+04
std,8.422074e+02,1.414708e+03,1.991921e+03,3.379610e+03
min,1.181600e+04,7.766000e+03,1.807700e+04,1.161500e+04
20%,1.301200e+04,9.352000e+03,1.953500e+04,1.695300e+04
50%,1.402200e+04,1.068000e+04,2.181900e+04,2.036800e+04
80%,1.455500e+04,1.205300e+04,2.316200e+04,2.287400e+04
max,1.646600e+04,1.505000e+04,2.648200e+04,2.758800e+04


In [5]:
# Suppression de la colonne "LS7" car ne contient que des valeurs nulles.
# Suppression de la colonne "Fichier" qui ne donne aucune information sur le score. 
# Suppression de la colonne "S13" car corrélée à la colonne S12

data_center_random = data_center_random.drop(['S6','S13', 'S14'], axis=1)
data_center_cluster = data_center_cluster.drop(['S6','S13', 'S14'], axis=1)
data_coin_random = data_coin_random.drop(['S6','S13', 'S14'], axis=1)
data_coin_cluster = data_coin_cluster.drop([ 'S6','S13', 'S14'], axis=1)

## Premier choix le 80:20 

In [8]:
# dans un premier temps les bonnes solutions seront les scores les 20% plus faibles 

In [9]:
data_center_random.Score = pd.cut(data_center_random.Score,bins=[0,13012,16466],labels=[0, 1])
data_center_cluster.Score = pd.cut(data_center_cluster.Score,bins=[0,9352,15050],labels=[0, 1])
data_coin_random.Score = pd.cut(data_coin_random.Score,bins=[0,19535,26482],labels=[0, 1])
data_coin_cluster.Score = pd.cut(data_coin_cluster.Score,bins=[0,16953,27588],labels=[0, 1])

# 0: Pour les 20% des scores les plus faibles donc les plus performants
# 1: pour les 80% restants 

## Second choix le 80:50:20

In [6]:
data_center_random.Score = pd.cut(data_center_random.Score,bins=[0,13012,14022,14555,16466],labels=[1, 2, 3, 4])
data_center_cluster.Score = pd.cut(data_center_cluster.Score,bins=[0,9352,10680,12053,15050],labels=[1, 2, 3, 4])
data_coin_random.Score = pd.cut(data_coin_random.Score,bins=[0,19535,21819,23162,26482],labels=[1, 2, 3, 4])
data_coin_cluster.Score = pd.cut(data_coin_cluster.Score,bins=[0,16953,20368,22874,27588],labels=[1, 2, 3, 4])

# 1: 20% meilleurs
# 2: 20-50% meilleurs 
# 3: 50-80% meilleurs
# 4: 80-100% meilleurs

In [7]:
# Création des jeux de données 

In [8]:
# Sélection d'un jeu de données parmis les 26/27 présent pour chaque configuration. 
# Le jeu de donnée est sélectionné au hasard. 

# Selection de 4 jeu de données 

data_center_random_01 = data_center_random.loc["01"]
data_center_cluster_01 = data_center_cluster.loc["01"]
data_coin_random_01 = data_coin_random.loc["01"]
data_coin_cluster_01 = data_coin_cluster.loc["01"]

data_center_random_05 = data_center_random.loc["05"]
data_center_cluster_05 = data_center_cluster.loc["05"]
data_coin_random_05 = data_coin_random.loc["05"]
data_coin_cluster_05 = data_coin_cluster.loc["05"]

data_center_random_13 = data_center_random.loc["13"]
data_center_cluster_13 = data_center_cluster.loc["13"]
data_coin_random_13 = data_coin_random.loc["13"]
data_coin_cluster_13 = data_coin_cluster.loc["13"]

data_center_random_20 = data_center_random.loc["20"]
data_center_cluster_20 = data_center_cluster.loc["20"]
data_coin_random_20 = data_coin_random.loc["20"]
data_coin_cluster_20 = data_coin_cluster.loc["20"]

# Jeu issue d'un mélange

data_center_random_mix = data_center_random.sample(frac = 0.04)
data_center_cluster_mix = data_center_cluster.sample(frac = 0.04)
data_coin_random_mix = data_coin_random.sample(frac = 0.04)
data_coin_cluster_mix = data_coin_cluster.sample(frac = 0.04)

In [9]:
# Séparation du jeu de donnée 01
X_CeR_01 = data_center_random_01.iloc[:,1:]
y_CeR_01 = data_center_random_01.iloc[:,0]

X_CeC_01 = data_center_cluster_01.iloc[:,1:]
y_CeC_01 = data_center_cluster_01.iloc[:,0]

X_CoR_01 = data_coin_random_01.iloc[:,1:]
y_CoR_01 = data_coin_random_01.iloc[:,0]

X_CoC_01 = data_coin_cluster_01.iloc[:,1:]
y_CoC_01 = data_coin_cluster_01.iloc[:,0]

X_train_CeR_01, X_test_CeR_01, y_train_CeR_01, y_test_CeR_01 = train_test_split(X_CeR_01, y_CeR_01, train_size = 0.8)
X_train_CeC_01, X_test_CeC_01, y_train_CeC_01, y_test_CeC_01 = train_test_split(X_CeC_01, y_CeC_01, train_size = 0.8)
X_train_CoR_01, X_test_CoR_01, y_train_CoR_01 ,y_test_CoR_01 = train_test_split(X_CoR_01, y_CoR_01, train_size = 0.8)
X_train_CoC_01, X_test_CoC_01, y_train_CoC_01, y_test_CoC_01 = train_test_split(X_CoC_01, y_CoC_01, train_size = 0.8)

In [10]:
# Séparation du jeu de donnée 05
X_CeR_05 = data_center_random_05.iloc[:,1:]
y_CeR_05 = data_center_random_05.iloc[:,0]

X_CeC_05 = data_center_cluster_05.iloc[:,1:]
y_CeC_05 = data_center_cluster_05.iloc[:,0]

X_CoR_05 = data_coin_random_05.iloc[:,1:]
y_CoR_05 = data_coin_random_05.iloc[:,0]

X_CoC_05 = data_coin_cluster_05.iloc[:,1:]
y_CoC_05 = data_coin_cluster_05.iloc[:,0]

X_train_CeR_05, X_test_CeR_05, y_train_CeR_05, y_test_CeR_05 = train_test_split(X_CeR_05, y_CeR_05, train_size = 0.8)
X_train_CeC_05, X_test_CeC_05, y_train_CeC_05, y_test_CeC_05 = train_test_split(X_CeC_05, y_CeC_05, train_size = 0.8)
X_train_CoR_05, X_test_CoR_05, y_train_CoR_05 ,y_test_CoR_05 = train_test_split(X_CoR_05, y_CoR_05, train_size = 0.8)
X_train_CoC_05, X_test_CoC_05, y_train_CoC_05, y_test_CoC_05 = train_test_split(X_CoC_05, y_CoC_05, train_size = 0.8)

In [11]:
# Séparation du jeu de donnée 13
X_CeR_13 = data_center_random_13.iloc[:,1:]
y_CeR_13 = data_center_random_13.iloc[:,0]

X_CeC_13 = data_center_cluster_13.iloc[:,1:]
y_CeC_13 = data_center_cluster_13.iloc[:,0]

X_CoR_13 = data_coin_random_13.iloc[:,1:]
y_CoR_13 = data_coin_random_13.iloc[:,0]

X_CoC_13 = data_coin_cluster_13.iloc[:,1:]
y_CoC_13 = data_coin_cluster_13.iloc[:,0]

X_train_CeR_13, X_test_CeR_13, y_train_CeR_13, y_test_CeR_13 = train_test_split(X_CeR_13, y_CeR_13, train_size = 0.8)
X_train_CeC_13, X_test_CeC_13, y_train_CeC_13, y_test_CeC_13 = train_test_split(X_CeC_13, y_CeC_13, train_size = 0.8)
X_train_CoR_13, X_test_CoR_13, y_train_CoR_13 ,y_test_CoR_13 = train_test_split(X_CoR_13, y_CoR_13, train_size = 0.8)
X_train_CoC_13, X_test_CoC_13, y_train_CoC_13, y_test_CoC_13 = train_test_split(X_CoC_13, y_CoC_13, train_size = 0.8)

In [12]:
# Séparation du jeu de donnée 20
X_CeR_20 = data_center_random_20.iloc[:,1:]
y_CeR_20 = data_center_random_20.iloc[:,0]

X_CeC_20 = data_center_cluster_20.iloc[:,1:]
y_CeC_20 = data_center_cluster_20.iloc[:,0]

X_CoR_20 = data_coin_random_20.iloc[:,1:]
y_CoR_20 = data_coin_random_20.iloc[:,0]

X_CoC_20 = data_coin_cluster_20.iloc[:,1:]
y_CoC_20 = data_coin_cluster_20.iloc[:,0]

X_train_CeR_20, X_test_CeR_20, y_train_CeR_20, y_test_CeR_20 = train_test_split(X_CeR_20, y_CeR_20, train_size = 0.8)
X_train_CeC_20, X_test_CeC_20, y_train_CeC_20, y_test_CeC_20 = train_test_split(X_CeC_20, y_CeC_20, train_size = 0.8)
X_train_CoR_20, X_test_CoR_20, y_train_CoR_20 ,y_test_CoR_20 = train_test_split(X_CoR_20, y_CoR_20, train_size = 0.8)
X_train_CoC_20, X_test_CoC_20, y_train_CoC_20, y_test_CoC_20 = train_test_split(X_CoC_20, y_CoC_20, train_size = 0.8)

In [13]:
# Séparation du jeu de donnée mix
X_CeR_mix = data_center_random_mix.iloc[:,1:]
y_CeR_mix = data_center_random_mix.iloc[:,0]

X_CeC_mix = data_center_cluster_mix.iloc[:,1:]
y_CeC_mix = data_center_cluster_mix.iloc[:,0]

X_CoR_mix = data_coin_random_mix.iloc[:,1:]
y_CoR_mix = data_coin_random_mix.iloc[:,0]

X_CoC_mix = data_coin_cluster_mix.iloc[:,1:]
y_CoC_mix = data_coin_cluster_mix.iloc[:,0]

X_train_CeR_mix, X_test_CeR_mix, y_train_CeR_mix, y_test_CeR_mix = train_test_split(X_CeR_mix, y_CeR_mix, train_size = 0.8)
X_train_CeC_mix, X_test_CeC_mix, y_train_CeC_mix, y_test_CeC_mix = train_test_split(X_CeC_mix, y_CeC_mix, train_size = 0.8)
X_train_CoR_mix, X_test_CoR_mix, y_train_CoR_mix ,y_test_CoR_mix = train_test_split(X_CoR_mix, y_CoR_mix, train_size = 0.8)
X_train_CoC_mix, X_test_CoC_mix, y_train_CoC_mix, y_test_CoC_mix = train_test_split(X_CoC_mix, y_CoC_mix, train_size = 0.8)

# Regression logistique

In [19]:
from sklearn import linear_model

In [20]:
model_RegLog_CeR = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto')

# Dataset mix

model_RegLog_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_RegLog_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_RegLog_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_RegLog_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_RegLog_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_RegLog_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_RegLog_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_RegLog_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_RegLog_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_RegLog_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_RegLog_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [21]:
model_RegLog_CeC = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto')

# Dataset mix

model_RegLog_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_RegLog_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_RegLog_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_RegLog_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_RegLog_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_RegLog_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_RegLog_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_RegLog_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_RegLog_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_RegLog_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_RegLog_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [22]:
model_RegLog_CoR = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto')

# Dataset mix

model_RegLog_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_RegLog_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_RegLog_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_RegLog_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_RegLog_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_RegLog_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_RegLog_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_RegLog_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_RegLog_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_RegLog_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_RegLog_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [23]:
model_RegLog_CoC = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto')

# Dataset mix

model_RegLog_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_RegLog_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_RegLog_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_RegLog_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_RegLog_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_RegLog_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_RegLog_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_RegLog_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_RegLog_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_RegLog_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_RegLog_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [24]:
data = {
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_mix,Center_cluster_mix,Coin_random_mix,Coin_cluster_mix
Score train_01,0.954762,1.000000,0.800271,0.605705
Score test_01,0.957706,1.000000,0.801895,0.603226
Score train_05,1.000000,1.000000,1.000000,1.000000
Score test_05,1.000000,1.000000,1.000000,1.000000
Score train_13,0.675397,0.942773,0.996032,1.000000
Score test_13,0.677385,0.943466,0.995603,1.000000
Score train_20,1.000000,1.000000,1.000000,1.000000
Score test_20,1.000000,1.000000,1.000000,1.000000
Score train_mix,0.900561,0.943682,0.965681,0.967006
Score test_mix,0.901941,0.945349,0.965380,0.967362


In [25]:
# les résultats sont bien meilleurs que lors de la regression 
# le fait de classifier les resultats possibles rend la tache plus simple 

# Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
model_DTC_CeR = DecisionTreeClassifier(random_state=42)
# Dataset mix

model_DTC_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_DTC_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_DTC_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_DTC_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_DTC_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_DTC_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_DTC_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_DTC_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_DTC_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_DTC_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_DTC_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [16]:
model_DTC_CeC = DecisionTreeClassifier(random_state=42)

# Dataset mix

model_DTC_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_DTC_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_DTC_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_DTC_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_DTC_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_DTC_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_DTC_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_DTC_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_DTC_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_DTC_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_DTC_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [17]:
model_DTC_CoR = DecisionTreeClassifier(random_state=42)

# Dataset mix

model_DTC_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_DTC_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_DTC_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_DTC_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_DTC_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_DTC_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_DTC_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_DTC_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_DTC_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_DTC_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_DTC_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [18]:
model_DTC_CoC = DecisionTreeClassifier(random_state=42)

# Dataset mix

model_DTC_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_DTC_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_DTC_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_DTC_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_DTC_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_DTC_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_DTC_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_DTC_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_DTC_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_DTC_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_DTC_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [19]:
data = {
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_mix,Center_cluster_mix,Coin_random_mix,Coin_cluster_mix
Score train_01,0.981857,0.823759,0.827761,0.933142
Score test_01,0.983653,0.817955,0.822129,0.933638
Score train_05,0.897892,0.935519,0.897766,0.998542
Score test_05,0.901750,0.940381,0.896677,0.998502
Score train_13,0.799531,0.999446,0.995308,1.000000
Score test_13,0.798190,0.999510,0.994923,1.000000
Score train_20,0.809131,1.000000,1.000000,1.000000
Score test_20,0.811556,1.000000,1.000000,1.000000
Score train_mix,1.000000,1.000000,1.000000,1.000000
Score test_mix,0.891939,0.948074,0.946136,0.978736


# KNNClassifier

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
i = 5

In [22]:
model_KNN5_CeR = KNeighborsClassifier(n_neighbors=i)
# Dataset mix

model_KNN5_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_KNN5_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_KNN5_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_KNN5_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_KNN5_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_KNN5_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_KNN5_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_KNN5_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_KNN5_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_KNN5_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_KNN5_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [23]:
model_KNN5_CeC = KNeighborsClassifier(n_neighbors=i)

# Dataset mix

model_KNN5_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_KNN5_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_KNN5_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_KNN5_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_KNN5_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_KNN5_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_KNN5_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_KNN5_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_KNN5_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_KNN5_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_KNN5_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [24]:
model_KNN5_CoR = KNeighborsClassifier(n_neighbors=i)

# Dataset mix

model_KNN5_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_KNN5_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_KNN5_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_KNN5_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_KNN5_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_KNN5_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_KNN5_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_KNN5_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_KNN5_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_KNN5_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_KNN5_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [25]:
model_KNN5_CoC = KNeighborsClassifier(n_neighbors=i)

# Dataset mix

model_KNN5_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_KNN5_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_KNN5_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_KNN5_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_KNN5_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_KNN5_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_KNN5_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_KNN5_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_KNN5_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_KNN5_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_KNN5_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [26]:
data = {
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_mix,Center_cluster_mix,Coin_random_mix,Coin_cluster_mix
Score train_01,0.968385,0.829744,0.817398,0.950807
Score test_01,0.968604,0.829925,0.813457,0.950075
Score train_05,0.892740,0.952328,0.897535,0.998532
Score test_05,0.897177,0.955510,0.896621,0.998305
Score train_13,0.740770,1.000000,0.998901,1.000000
Score test_13,0.738945,1.000000,0.998999,1.000000
Score train_20,0.724609,0.999987,0.974303,1.000000
Score test_20,0.721629,1.000000,0.973821,1.000000
Score train_mix,0.885704,0.965521,0.958189,0.985670
Score test_mix,0.835814,0.951328,0.942783,0.978796


# SVM

In [46]:
from sklearn import svm

In [47]:
model_SVM_CeR = svm.SVC()
# Dataset mix

model_SVM_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_SVM_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_SVM_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_SVM_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_SVM_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_SVM_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_SVM_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_SVM_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_SVM_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_SVM_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_SVM_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [48]:
model_SVM_CeC = svm.SVC()

# Dataset mix

model_SVM_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_SVM_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_SVM_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_SVM_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_SVM_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_SVM_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_SVM_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_SVM_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_SVM_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_SVM_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_SVM_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [49]:
model_SVM_CoR = svm.SVC()

# Dataset mix

model_SVM_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_SVM_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_SVM_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_SVM_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_SVM_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_SVM_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_SVM_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_SVM_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_SVM_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_SVM_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_SVM_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [50]:
model_SVM_CoC = svm.SVC()

# Dataset mix

model_SVM_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_SVM_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_SVM_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_SVM_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_SVM_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_SVM_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_SVM_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_SVM_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_SVM_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_SVM_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_SVM_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [51]:
data = {
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_mix,Center_cluster_mix,Coin_random_mix,Coin_cluster_mix
Score train_01,0.999373,1.000000,0.631794,0.423120
Score test_01,0.999741,1.000000,0.634960,0.421293
Score train_05,1.000000,1.000000,0.999839,1.000000
Score test_05,1.000000,1.000000,0.999748,1.000000
Score train_13,0.771107,0.991294,0.998892,1.000000
Score test_13,0.776593,0.991470,0.998749,1.000000
Score train_20,1.000000,1.000000,1.000000,1.000000
Score test_20,1.000000,1.000000,1.000000,1.000000
Score train_mix,0.889843,0.946010,0.949353,0.951390
Score test_mix,0.887849,0.947203,0.948855,0.949909


# Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
model_RFC_CeR = RandomForestClassifier(max_depth=4, random_state=0)
# Dataset mix

model_RFC_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_RFC_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_RFC_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_RFC_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_RFC_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_RFC_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_RFC_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_RFC_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_RFC_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_RFC_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_RFC_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [34]:
model_RFC_CeC = RandomForestClassifier(max_depth=4, random_state=0)

# Dataset mix

model_RFC_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_RFC_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_RFC_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_RFC_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_RFC_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_RFC_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_RFC_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_RFC_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_RFC_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_RFC_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_RFC_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [35]:
model_RFC_CoR = RandomForestClassifier(max_depth=4, random_state=0)

# Dataset mix

model_RFC_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_RFC_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_RFC_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_RFC_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_RFC_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_RFC_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_RFC_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_RFC_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_RFC_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_RFC_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_RFC_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [36]:
model_RFC_CoC = RandomForestClassifier(max_depth=4, random_state=0)

# Dataset mix

model_RFC_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_RFC_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_RFC_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_RFC_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_RFC_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_RFC_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_RFC_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_RFC_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_RFC_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_RFC_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_RFC_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [37]:
data = {
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_mix,Center_cluster_mix,Coin_random_mix,Coin_cluster_mix
Score train_01,0.955994,0.503482,0.781051,0.857076
Score test_01,0.955371,0.502565,0.777669,0.855715
Score train_05,0.411579,0.729096,0.896941,0.998542
Score test_05,0.416093,0.732074,0.897992,0.998305
Score train_13,0.731169,0.995239,0.997158,1.000000
Score test_13,0.729035,0.994541,0.997068,1.000000
Score train_20,0.777574,1.000000,0.999970,1.000000
Score test_20,0.778163,1.000000,0.999970,1.000000
Score train_mix,0.779649,0.867750,0.908697,0.833510
Score test_mix,0.777077,0.868329,0.908737,0.832819


# Neural network

In [52]:
from sklearn.neural_network import MLPClassifier

In [54]:
model_MLPC_CeR = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# Dataset mix

model_MLPC_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_MLPC_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_MLPC_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_MLPC_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_MLPC_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_MLPC_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_MLPC_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_MLPC_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_MLPC_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_MLPC_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_MLPC_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [55]:
model_MLPC_CeC = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# Dataset mix

model_MLPC_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_MLPC_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_MLPC_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_MLPC_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_MLPC_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_MLPC_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_MLPC_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_MLPC_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_MLPC_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_MLPC_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_MLPC_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [56]:
model_MLPC_CoR = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# Dataset mix

model_MLPC_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_MLPC_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_MLPC_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_MLPC_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_MLPC_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_MLPC_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_MLPC_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_MLPC_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_MLPC_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_MLPC_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_MLPC_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [57]:
model_MLPC_CoC = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# Dataset mix

model_MLPC_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_MLPC_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_MLPC_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_MLPC_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_MLPC_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_MLPC_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_MLPC_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_MLPC_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_MLPC_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_MLPC_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_MLPC_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [58]:
data = {
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_mix,Center_cluster_mix,Coin_random_mix,Coin_cluster_mix
Score train_01,1.000000,1.000000,0.636562,0.576906
Score test_01,1.000000,1.000000,0.638804,0.578707
Score train_05,1.000000,1.000000,1.000000,1.000000
Score test_05,1.000000,1.000000,1.000000,1.000000
Score train_13,0.792934,1.000000,0.000822,1.000000
Score test_13,0.797005,1.000000,0.001001,1.000000
Score train_20,1.000000,1.000000,1.000000,1.000000
Score test_20,1.000000,1.000000,1.000000,1.000000
Score train_mix,0.801158,0.800030,0.802879,0.800054
Score test_mix,0.799251,0.798388,0.800375,0.797157
